## Final Project: PC3 Data Set

In [86]:
library(class)
library (ROCR)
#library (iterativeBMA)
library (e1071)

In [87]:
data <- read.delim("/Users/spoorthy/Desktop/Bio_info_final_project/CAMDA-master/PC3_RMA_labels.txt")

## Data Preperation
#####    First 190 rows are training data and the remaining 86 data are test data.  Test data does not have label data.  Last column is label column
#####  Split the data into training_data and training_data_label

In [88]:
na.ind.labels <- which(is.na(data$Label))
length(na.ind.labels)


[1] 86

## Training data

In [89]:
# Training data
training_data  <- data[-na.ind.labels,-22278]
dim(training_data)
training_data[1:5,22270:22277]

[1]   190 22277

AFFX.r2.P1.cre.3_at,AFFX.r2.P1.cre.5_at,AFFX.ThrX.3_at,AFFX.ThrX.5_at,AFFX.ThrX.M_at,AFFX.TrpnX.3_at,AFFX.TrpnX.5_at,AFFX.TrpnX.M_at
14.58883,14.41516,5.328093,4.787588,4.833590,4.614942,4.537976,4.664314
14.18400,13.81357,5.479934,5.608054,5.216571,5.422156,4.883759,5.329954
14.11321,13.65796,5.513578,5.584934,5.310678,5.523111,5.046227,5.481926
13.80475,13.49202,4.937254,4.683656,4.591943,4.583175,4.377955,4.941050
12.47760,12.46260,3.244258,3.011891,2.818193,2.878322,2.825760,3.208905


## Labels for training data

In [90]:
training_data_label  <- data.frame(data[-na.ind.labels,22278])
dim(training_data_label)
training_data_label

[1] 190   1

data..na.ind.labels..22278.
1
1
0
1
0
0
1
1
1
1


## Test data

In [91]:
test_data  <- data[na.ind.labels,-22278]
dim(test_data)
test_data[1:5,22270:22277]

[1]    86 22277

,AFFX.r2.P1.cre.3_at,AFFX.r2.P1.cre.5_at,AFFX.ThrX.3_at,AFFX.ThrX.5_at,AFFX.ThrX.M_at,AFFX.TrpnX.3_at,AFFX.TrpnX.5_at,AFFX.TrpnX.M_at
191,14.54871,14.26104,5.284418,5.042819,4.929295,4.999701,4.769721,5.023306
192,14.93217,14.35377,4.876677,4.543831,4.243698,4.322675,4.380935,4.400256
193,12.87484,12.69556,4.505898,4.493102,4.284225,4.577340,4.177343,4.416013
194,13.22302,12.70105,4.730601,4.619996,4.146032,4.339825,4.206135,4.861587
195,13.59059,13.23581,4.987098,5.043222,4.792481,5.024211,4.594230,5.277112


## Performing K fold Cross Validation 

### KNN

In [102]:
#KNN with K=10 with feature selection with high corelation
Highcorrelation10KNN = function(xDataRaw, yDataRaw,kFolds,repeats) {

  folds = cut(seq(1, nrow(xDataRaw)), breaks=kFolds, labels=FALSE);
  yDataRaw = yDataRaw[rownames(xDataRaw), , drop=FALSE]

  #Creat prediction dataframe to save result of each round
  predictionRound<-NULL

  for (r in 1:repeats) {
    # Feature Selection based on correaltion
    #correlationVector = abs(apply(xDataRaw, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
    correlationVector = abs(apply (t(xDataRaw), 1, function(x){cor (x, yDataRaw)}))
    correlatedColumns = names(correlationVector[correlationVector >= quantile(correlationVector, prob=1-5/100)])

    # Shuffle Data
    xData = xDataRaw[sample(nrow(xDataRaw)), correlatedColumns, drop=FALSE];
    # print(xData)
    yData = yDataRaw[rownames(xData), , drop=FALSE];

    #Creat prediction dataframe to save result of each round
    predictionFold<-NULL

    for (k in 1:kFolds) {
      testIndexes = which(folds == k, arr.ind=TRUE)
      xTestData = xData[testIndexes, , drop=FALSE]
      xTrainData = xData[-testIndexes, , drop=FALSE]
      yTrainData = yData[-testIndexes, , drop=FALSE]
      yTestData = yData[testIndexes, , drop=FALSE]
        
      testClass<-xData[testIndexes, c(1,1,1)]
     
      names(testClass)[2]<-"round"
      testClass$round<-r
      names(testClass)[3]<-"fold"
      testClass$fold<-k

      testClass$p <-knn(train=xTrainData,test=xTestData,cl=unlist(yTrainData), k=10)

      predictionFold <- rbind(predictionFold,testClass)
    }
    predictionRound <- rbind(predictionRound,predictionFold)
  }
  return(predictionRound)
}

In [103]:
#KNN with K=10 with feature selection with Low corelation
Lowcorrelation10KNN = function(xDataRaw, yDataRaw,kFolds,repeats) {


  folds = cut(seq(1, nrow(xDataRaw)), breaks=kFolds, labels=FALSE);
  yDataRaw = yDataRaw[rownames(xDataRaw), , drop=FALSE]

  #Creat prediction dataframe to save result of each round
  predictionRound<-NULL

  for (r in 1:repeats) {
    # Feature Selection based on correaltion
    #correlationVector = abs(apply(xDataRaw, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
    correlationVector = abs(apply (t(xDataRaw), 1, function(x){cor (x, yDataRaw)}))
    correlatedColumns = names(correlationVector[correlationVector >= quantile(correlationVector, prob=1-25/100)])

    # Shuffle Data
    xData = xDataRaw[sample(nrow(xDataRaw)), correlatedColumns, drop=FALSE];
    # print(xData)
    yData = yDataRaw[rownames(xData), , drop=FALSE];

    #Creat prediction dataframe to save result of each round
    predictionFold<-NULL

    for (k in 1:kFolds) {
      testIndexes = which(folds == k, arr.ind=TRUE)
      xTestData = xData[testIndexes, , drop=FALSE]
      xTrainData = xData[-testIndexes, , drop=FALSE]
      yTrainData = yData[-testIndexes, , drop=FALSE]
      yTestData = yData[testIndexes, , drop=FALSE]
        
      testClass<-xData[testIndexes, c(1,1,1)]
     
      names(testClass)[2]<-"round"
      testClass$round<-r
      names(testClass)[3]<-"fold"
      testClass$fold<-k

      testClass$p <-knn(train=xTrainData,test=xTestData,cl=unlist(yTrainData), k=10)

      predictionFold <- rbind(predictionFold,testClass)
    }
    predictionRound <- rbind(predictionRound,predictionFold)
  }
  return(predictionRound)
}

## SVM

In [104]:
#SVM with K=10 with feature selection with Low corelation
Lowcorrelation10SVM = function(xDataRaw, yDataRaw,kFolds,repeats) {

  folds = cut(seq(1, nrow(xDataRaw)), breaks=kFolds, labels=FALSE);
  yDataRaw = yDataRaw[rownames(xDataRaw), , drop=FALSE]

  #Creat prediction dataframe to save result of each round
  predictionRound<-NULL

  for (r in 1:repeats) {
    # Feature Selection based on correaltion
    #correlationVector = abs(apply(xDataRaw, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
    correlationVector = abs(apply (t(xDataRaw), 1, function(x){cor (x, yDataRaw)}))
    correlatedColumns = names(correlationVector[correlationVector >= quantile(correlationVector, prob=1-25/100)])

    # Shuffle Data
    xData = xDataRaw[sample(nrow(xDataRaw)), correlatedColumns, drop=FALSE];
    # print(xData)
    yData = yDataRaw[rownames(xData), , drop=FALSE];

    #Creat prediction dataframe to save result of each round
    predictionFold<-NULL

    for (k in 1:kFolds) {
      testIndexes = which(folds == k, arr.ind=TRUE)
      xTestData = xData[testIndexes, , drop=FALSE]
      xTrainData = xData[-testIndexes, , drop=FALSE]
      yTrainData = yData[-testIndexes, , drop=FALSE]
      yTestData = yData[testIndexes, , drop=FALSE]
        
      testClass<-xData[testIndexes, c(1,1,1)]
     
      names(testClass)[2]<-"round"
      testClass$round<-r
      names(testClass)[3]<-"fold"
      testClass$fold<-k

      ret.svm <- svm (xTrainData, unlist(yTrainData))
      testClass$p <- predict (ret.svm, xTestData)
        
      testClass$p <- round(testClass$p)
        
      predictionFold <- rbind(predictionFold,testClass)
    }
    predictionRound <- rbind(predictionRound,predictionFold)
  }
  return(predictionRound)
}

In [105]:
#SVM with K=10 with feature selection with High corelation
Highcorrelation10SVM = function(xDataRaw, yDataRaw,kFolds,repeats) {

  folds = cut(seq(1, nrow(xDataRaw)), breaks=kFolds, labels=FALSE);
  yDataRaw = yDataRaw[rownames(xDataRaw), , drop=FALSE]

  #Creat prediction dataframe to save result of each round
  predictionRound<-NULL

  for (r in 1:repeats) {
    # Feature Selection based on correaltion
    #correlationVector = abs(apply(xDataRaw, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
    correlationVector = abs(apply (t(xDataRaw), 1, function(x){cor (x, yDataRaw)}))
    correlatedColumns = names(correlationVector[correlationVector >= quantile(correlationVector, prob=1-10/100)])

    # Shuffle Data
    xData = xDataRaw[sample(nrow(xDataRaw)), correlatedColumns, drop=FALSE];
    # print(xData)
    yData = yDataRaw[rownames(xData), , drop=FALSE];

    #Creat prediction dataframe to save result of each round
    predictionFold<-NULL

    for (k in 1:kFolds) {
      testIndexes = which(folds == k, arr.ind=TRUE)
      xTestData = xData[testIndexes, , drop=FALSE]
      xTrainData = xData[-testIndexes, , drop=FALSE]
      yTrainData = yData[-testIndexes, , drop=FALSE]
      yTestData = yData[testIndexes, , drop=FALSE]
        
      testClass<-xData[testIndexes, c(1,1,1)]
     
      names(testClass)[2]<-"round"
      testClass$round<-r
      names(testClass)[3]<-"fold"
      testClass$fold<-k

      ret.svm <- svm (xTrainData, unlist(yTrainData))
      testClass$p <- predict (ret.svm, xTestData)
        
      testClass$p <- round(testClass$p)
        
      predictionFold <- rbind(predictionFold,testClass)
    }
    predictionRound <- rbind(predictionRound,predictionFold)
  }
  return(predictionRound)
}

In [106]:
#SVM with K=10 with feature selection with High corelation
Correlation0.001SVM = function(xDataRaw, yDataRaw,kFolds,repeats) {

  folds = cut(seq(1, nrow(xDataRaw)), breaks=kFolds, labels=FALSE);
  yDataRaw = yDataRaw[rownames(xDataRaw), , drop=FALSE]

  #Creat prediction dataframe to save result of each round
  predictionRound<-NULL

  for (r in 1:repeats) {
    # Feature Selection based on correaltion
    #correlationVector = abs(apply(xDataRaw, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
    correlationVector = abs(apply (t(xDataRaw), 1, function(x){cor (x, yDataRaw)}))
    correlatedColumns = names(correlationVector[correlationVector >= quantile(correlationVector, prob=0.001)])

    # Shuffle Data
    xData = xDataRaw[sample(nrow(xDataRaw)), correlatedColumns, drop=FALSE];
    # print(xData)
    yData = yDataRaw[rownames(xData), , drop=FALSE];

    #Creat prediction dataframe to save result of each round
    predictionFold<-NULL

    for (k in 1:kFolds) {
      testIndexes = which(folds == k, arr.ind=TRUE)
      xTestData = xData[testIndexes, , drop=FALSE]
      xTrainData = xData[-testIndexes, , drop=FALSE]
      yTrainData = yData[-testIndexes, , drop=FALSE]
      yTestData = yData[testIndexes, , drop=FALSE]
        
      testClass<-xData[testIndexes, c(1,1,1)]
     
      names(testClass)[2]<-"round"
      testClass$round<-r
      names(testClass)[3]<-"fold"
      testClass$fold<-k

      ret.svm <- svm (xTrainData, unlist(yTrainData))
      testClass$p <- predict (ret.svm, xTestData)
        
      testClass$p <- round(testClass$p)
        
      predictionFold <- rbind(predictionFold,testClass)
    }
    predictionRound <- rbind(predictionRound,predictionFold)
  }
  return(predictionRound)
}

## Random Forest

In [98]:
source("https://bioconductor.org/biocLite.R")
biocLite(c("randomForest"))
library (randomForest)

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.4 (2018-03-15).
Installing package(s) ‘randomForest’



The downloaded binary packages are in
	/var/folders/0z/bnd8yq6d0d15v_nlpw_fd2pm0000gn/T//RtmpgMsA57/downloaded_packages


Old packages: 'bit', 'data.table', 'dendextend', 'dplyr', 'FactoMineR',
  'ggrepel', 'ggsci', 'httpuv', 'inline', 'IRdisplay', 'lambda.r', 'later',
  'Matrix', 'nlme', 'openxlsx', 'pillar', 'Rcpp', 'repr', 'rjson', 'RMySQL',
  'rrcov', 'RSQLite', 'shiny', 'stringr', 'survival', 'tidyr', 'utf8', 'yaml'


In [107]:
#Random with K=10 with feature selection.
randonForest = function(xDataRaw, yDataRaw,kFolds,repeats) {

  folds = cut(seq(1, nrow(xDataRaw)), breaks=kFolds, labels=FALSE);
  yDataRaw = yDataRaw[rownames(xDataRaw), , drop=FALSE]

  #Creat prediction dataframe to save result of each round
  predictionRound<-NULL

  for (r in 1:repeats) {
    # Feature Selection based on correaltion
    #correlationVector = abs(apply(xDataRaw, 2, function(x) {cor(unlist(x), unlist(yDataRaw))}))
    correlationVector = abs(apply (t(xDataRaw), 1, function(x){cor (x, yDataRaw)}))
    correlatedColumns = names(correlationVector[correlationVector >= quantile(correlationVector, prob=1-5/100)])

    # Shuffle Data
    xData = xDataRaw[sample(nrow(xDataRaw)), correlatedColumns, drop=FALSE];
    # print(xData)
    yData = yDataRaw[rownames(xData), , drop=FALSE];

    #Creat prediction dataframe to save result of each round
    predictionFold<-NULL

    for (k in 1:kFolds) {
      testIndexes = which(folds == k, arr.ind=TRUE)
      xTestData = xData[testIndexes, , drop=FALSE]
      xTrainData = xData[-testIndexes, , drop=FALSE]
      yTrainData = yData[-testIndexes, , drop=FALSE]
      yTestData = yData[testIndexes, , drop=FALSE]
        
      testClass<-xData[testIndexes, c(1,1,1)]
     
      names(testClass)[2]<-"round"
      testClass$round<-r
      names(testClass)[3]<-"fold"
      testClass$fold<-k

       randforest <- randomForest(as.factor(unlist(yTrainData)) ~ ., xTrainData, ntree=500,importance=TRUE)
        testClass$p <- predict (randforest, xTestData)
        
      #testClass$p <- round(testClass$p)
        
      predictionFold <- rbind(predictionFold,testClass)
    }
    predictionRound <- rbind(predictionRound,predictionFold)
  }
  return(predictionRound)
}

In [108]:
# evalMethod - the evaluation method function to compute number of errors using mean and table function
# result - output from k-fold validation
# yData - trainign data labels

evalMethod <- function(result,yData)
{
    # round -get the number of rounds repeated on k-fold cross validation
    round = max(result$round)

    #variable to calculate prediction accuracy using 'mean' funciton
    accuracies<- c()
    
    # variable to store the prediction accuracy using 'table' function
    totalpredacc <- 0
    
    for(i in 1:round){
        
        # get the data from output for the round i
        evaluation.data<- subset(result, result$round==i ) 
        
        # use 'table' function to predict the model accuracy
        predtable <- table (evaluation.data$p, unname(unlist(training_data_label)))
        
        # compute the errors from the result that is obatined from 'table' function
        error=(predtable[1,2]+predtable[2,1])/length(evaluation.data$p)
        
        # compute the prediction by subtracting error from 1
        predacc = (1- error)*100
    
        # final matrix to hold the computed avaergae prediction accuracy
        # and the corresponding side effects name
        final <- yData[ncol(yData),c(1,1,1)]
        names(final)[1]<-"Method_Names"
        names(final)[2]<-"Prediction_Accuracy_TableFunction"
        names(final)[3]<-"Prediction_Accuracy_MeanFunction"
        
        # add prediction accuracy from each round
        totalpredacc <- totalpredacc + predacc
        
        # compute the prediction using mean function
        accuracy <- mean(evaluation.data$p == unlist(yData))
        accuracies <- c(accuracies, accuracy)
        
        }
    
    rownames(final)<-NULL
    # compute the average prediction accuracy from the calculated total prediction accuracy (table funciton)
    avgpredacc=totalpredacc/round
    
    # compute the average prediction accuracy from the calculated total prediction accuracy(mean function)
    averageAccuracy = mean(accuracies)
      
    
    final$Prediction_Accuracy_MeanFunction <- averageAccuracy*100
    final$Prediction_Accuracy_TableFunction <- avgpredacc
     
    return(final)    
} 

In [109]:
 set.seed(42)

 methodNames = c("Lowcorrelation10KNN","Highcorrelation10KNN","Lowcorrelation10SVM","Highcorrelation10SVM","Correlation0.001SVM","randonForest")
 methods = c(Lowcorrelation10KNN,Highcorrelation10KNN,Lowcorrelation10SVM,Highcorrelation10SVM,Correlation0.001SVM,randonForest)

  output <- NULL

for (method in methods) {

    result<-method(training_data, training_data_label,10,3)
    r <- evalMethod(result,training_data_label)
    output <- rbind(output,r)
    
}
output[1] <- methodNames
write.csv(output, file="Bio_Project_Result1.csv")